## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,50.11,50,0,11.50,clear sky
1,1,Bloemfontein,ZA,-29.1211,26.2140,81.88,34,20,6.91,few clouds
2,2,Aden,YE,12.7794,45.0367,78.10,55,20,4.00,few clouds
3,3,Kapaa,US,22.0752,-159.3190,66.18,94,0,5.75,clear sky
4,4,Haines Junction,CA,60.7522,-137.5108,-0.45,78,100,1.14,overcast clouds


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 60


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

In [16]:
# 4a. Determine if there are any empty rows.
print("NOT NULL: " + str(preferred_cities_df["City_ID"].notnull().sum()))
print("NULL: " + str(preferred_cities_df["City_ID"].isnull().sum()))
print("Total: " + str(preferred_cities_df["City_ID"].count()))

NOT NULL: 71
NULL: 0
Total: 71


In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Punta Arenas,CL,50.11,clear sky,-53.1500,-70.9167,
13,Mataura,NZ,48.47,overcast clouds,-46.1927,168.8643,
18,San Jose,US,54.25,few clouds,37.3394,-121.8950,
35,Ushuaia,AR,51.42,clear sky,-54.8000,-68.3000,
39,Tateyama,JP,48.13,few clouds,34.9833,139.8667,
43,Rota,ES,59.13,light rain,36.6224,-6.3586,
52,Bluff,NZ,48.79,broken clouds,-46.6000,168.3333,
57,Hobart,AU,59.25,overcast clouds,-42.8794,147.3294,
59,Kaitangata,NZ,52.93,overcast clouds,-46.2817,169.8464,
81,Victoria,HK,59.25,few clouds,22.2855,114.1577,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
print("Done.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Done.


In [20]:
print("Rows: " + str(len(hotel_df.index)))

# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"]==""].index)

print("Rows without blank hotels: " + str(len(clean_hotel_df.index)))


Rows: 71
Rows without blank hotels: 69


In [21]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Punta Arenas,CL,50.11,clear sky,-53.1500,-70.9167,Hotel Hain
13,Mataura,NZ,48.47,overcast clouds,-46.1927,168.8643,Ellie's Villa
18,San Jose,US,54.25,few clouds,37.3394,-121.8950,The Westin San Jose
35,Ushuaia,AR,51.42,clear sky,-54.8000,-68.3000,Albatross Hotel
39,Tateyama,JP,48.13,few clouds,34.9833,139.8667,Tateyama Seaside Hotel
...,...,...,...,...,...,...,...
640,Tura,IN,53.44,overcast clouds,25.5198,90.2201,Hotel RIKMAN Continental
655,Le Mans,FR,46.33,clear sky,48.0000,0.2000,ibis Styles Le Mans Centre Gare
659,Alice Springs,AU,58.55,clear sky,-23.7000,133.8833,Desert Palms Alice Springs
660,Petilia Policastro,IT,48.69,overcast clouds,39.1139,16.7855,Residenza D'Epoca Palazzo Verga


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

#plt.savefig("WeatherPy_vacation_map.png")

Figure(layout=FigureLayout(height='420px'))